In [ ]:
import os
from glob import glob

import earthpy as et
from earthpy import earthexplorer as etee
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge
from rioxarray import merge

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
chi_dir = os.path.join(data_dir, 'chicago-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'chicago-green-space', 'processed')

for a_dir in [chi_dir, ndvi_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

In [ ]:
chi_path = os.path.join(chi_dir, 'chicago-neighborhoods.shp')
if not os.path.exists(chi_path):
    chi_url = (
        "https://data.cityofchicago.org/api/geospatial/"
        "bbvz-uum9?method=export&format=Shapefile")
    gpd.read_file(chi_url).to_file(chi_path)
    
chi_gdf = gpd.read_file(chi_path).set_index('PRI_NEIGH').to_crs(4326)
# chi_gdf = gpd.read_file(chi_path).set_index('PRI_NEIGH').to_crs(4326)
chi_gdf

neigh_gdf = (
    chi_gdf
    .loc[['Humboldt Park', 'Lincoln Park']]
    )

neigh_gdf

In [ ]:
def load_and_merge_arrays(name):
  """
  Load in and merge downloaded arrays

  Parameters
  ==========
  name : str
    The name used to label the download

  Returns
  =======
  merged_da : rxr.DataArray 
    DataArray with the merged data
  """

  print(f'\nNeighborhood Name: {name}')
  data_path = os.path.join(
      et.io.HOME, et.io.DATA_NAME,
      name.lower().replace(' ', '-'))
  tif_paths = glob(os.path.join(data_path, '*.tif'))
  das = [rxr.open_rasterio(tp, masked=True) for tp in tif_paths]
  merged_da = rxrmerge.merge_arrays(das)
  return merged_da

merged_da = load_and_merge_arrays('Lincoln Park')

In [ ]:
def calculate_ndvi_statistics(gdf, da, stats_path, override=False):
  """
  Calculate NDVI, then summarize and save statistics

  Uses downloaded National Agricultural Imagery Program (NAIP)
  data. 

  Parameters
  ==========
  gdf : gpd.GeodataFrame
    One row tihe the neighborhood name and boundary
  da : rxr.DataArray
    Multispectral (NAIP) raster data 
  stats_path: pathlike
    The path to the statistics file to save results
  """
  name = str(gdf.index[0])
  print(f'Neighborhood name: {name}')

  file_is_empty = True
  if os.path.exists(stats_path):
    print('Stats file exists.')
    stats_df = pd.read_csv(stats_path)
    file_is_empty = len(stats_df)==0
    print(f'Stats file is empty? {file_is_empty}')

    if not file_is_empty:
      if (name in stats_df.neighborhood) and (not override):
        print('Stats already calculated. Skipping...')
        return
      
  reprojected_gdf = gdf.to_crs(da.rio.crs)

  naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
  naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)

  ndvi_da = (
    (da.sel(band=4) - da.sel(band=1))
    / (da.sel(band=4) + da.sel(band=1))
  )

  print('Writing stats to file')
  mode = 'w' if file_is_empty else 'a'
  pd.DataFrame(dict(
    neighborhood=[name],
    ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
    ndvi_mean=[float(ndvi_da.mean())]
  )).to_csv(stats_path, mode=mode, header=file_is_empty, index=False)

calculate_ndvi_statistics(
  chi_gdf.loc[['Lincoln Park']], merged_da, ndvi_stats_path)

In [ ]:

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

for neighborhood_name, details in neigh_gdf.iterrows():
  if not os.path.exists(ndvi_stats_path):
    print('NDVI statistics file does not exist...')
    ndvi_stats_df = pd.DataFrame()
  else:
    ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")

  if neighborhood_name in ndvi_stats_df.index:
    print('Neighborhood stats have already been calculated. Skipping')
    continue
    
  downloader = download_neighborhood_data(
      neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
  merged_da = load_and_merge_arrays(neighborhood_name)
  calculate_ndvi_statistics(
      chi_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
  
  shututil.rmtree(downloader.data_dir)

In [ ]:
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
chloropleth = gv.Polygons(
    chi_gdf.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_mean']
)
hv.save(chloropleth, 'chloropleth.png')